# SUBMISSION GENERATOR

## 1-IMPORTS

In [1]:
import json
import os
from submission_generator_tools import *

## 2-DEFINING USEFUL FUNCTIONS

In [2]:
########## HEADER ##########
# Config file description :

# network_learning_rate : float : No idea, but certainly linked to the back propagation ? Default : 0.0005.
# network_n_classes : int : number of labels in the output. Default : 2.
# network_dropout : float : between 0 and 1 : percentage of neurons we want to keep. Default : 0.75.
# network_depth : int : number of layers WARNING : factualy, there will be 2*network_depth layers. Default : 6.
# network_convolution_per_layer : list of int, length = network_depth : number of convolution per layer. Default : [1 for i in range(network_depth)].
# network_size_of_convolutions_per_layer : list of lists of int [number of layers[number_of_convolutions_per_layer]] : Describe the size of each convolution filter.
# Default : [[3 for k in range(network_convolution_per_layer[i])] for i in range(network_depth)].
# network_features_per_layer : list of lists of int [number of layers[number_of_convolutions_per_layer[2]] : Numer of different filters that are going to be used.
# Default : [[64 for k in range(network_convolution_per_layer[i])] for i in range(network_depth)]. WARNING ! network_features_per_layer[k][1] = network_features_per_layer[k+1][0].
# network_trainingset : string : describe the trainingset for the network.
# network_downsampling : string 'maxpooling' or 'convolution' : the downsampling method.
# network_thresholds : list of float in [0,1] : the thresholds for the ground truthes labels.
# network_weighted_cost : boolean : whether we use weighted cost for training or not.


## 3-GENERATING THE FILES

Defining the name of the files and models, and environment variables.

**IMPORTANT TO CHANGE AT EACH GENERATION**

In [3]:
server = 'Guillimin' # Choose between Guillimin and Helios

# Warning: model names must finish by "/"

path_models = '../models/'
path_data = '../../../trainingsets/'
    

Defining the test config files

In [4]:
config_list = {}

In [5]:
config_list.update(generate_dict( network_learning_rate = 0.0005,
	                     network_n_classes = 2,
	                     dropout = 0.75,
	                     structure = [[5,5,5], [3,3,3], [3,3,3], [3,3,3]],  
	                     features_augmentation = 'p10',
	                     network_first_num_features = 15,
	                     trainingset = 'SEM_2classes_reduced',
	                     downsampling = 'convolution',
	                     thresholds = [0, 0.5],
	                     weighted_cost = False,
                         batch_size = 8))

config_list.update(generate_dict( network_learning_rate = 0.0005,
	                     network_n_classes = 2,
	                     dropout = 0.75,
	                     structure = [[5,5,5], [5,5,5], [3,3,3], [3,3,3]],  
	                     features_augmentation = 'x2',
	                     network_first_num_features = 16,
	                     trainingset = 'SEM_2classes_reduced',
	                     downsampling = 'maxpooling',
	                     thresholds = [0, 0.5],
	                     weighted_cost = False,
                         batch_size = 8))



We now generate the bash file that will launch all submissions at once, as well as the required sh files.

In [9]:
file = open(path_models + "/global_submission.sh","w") # We create the general submission file at the root of the models folder

for config_name,config in config_list.iteritems():

    submission_filename = "submission.sh"
    if server == 'Helios':
        file.write('msub ')
    elif server == 'Guillimin':
        file.write('qsub ')
    file.write(str(config_name)+'/')
    file.write(str(submission_filename))
    if server == 'Guillimin':
        file.write(' -l nodes=1:gpus=1:exclusive_process') # To avoid a server error. Check that the number of nodes is the good one !
    file.write('\n')
    
    if server == 'Helios':
        generate_heliosjob(submission_filename, 'config_network.json', config,
                       path_trainingset = path_data+config['network_trainingset']+'/training/',
                       path_model = path_models+config_name
                      )
    elif server == 'Guillimin':
        generate_guilliminjob(submission_filename, 'config_network.json', config,
                       path_trainingset = path_data+config['network_trainingset']+'/training/',
                       path_model = path_models+config_name
                      )

file.close()